In [1]:
import pandas as pd

In [2]:
sbmt = pd.read_csv('../data/raw/submission.csv')
print(sbmt.shape)
sbmt.head()

(1695, 2)


,user_id,movie_id
0,16975709365084587137,0 0 0 0 0
1,10225505309438393880,0 0 0 0 0
2,337894458057354820,0 0 0 0 0
3,16389753737490268206,0 0 0 0 0
4,12377183607123393787,0 0 0 0 0


# top 5 pred

In [4]:
most_pop = [12304, 7735, 15285, 17518, 19026]

In [18]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in most_pop])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/top_5.csv', index = False)

# 5 from top 100

In [23]:
import pickle

with open('../data/processed/random_5_from_top100.pickle', 'rb') as f:
    preds = pickle.load(f)

In [25]:
predict = sbmt.copy()
predict = predict.drop(' movie_id', 1)

for ind, user in enumerate(sbmt.iterrows()):
    #print(user[1]['user_id'])
    
    pred = ' '.join([str(i) for i in preds[user[1]['user_id']]])
    
    predict.loc[ind, 'movie_id'] = pred
    
predict.to_csv('../data/submits/random_5_from_top_100.csv', index = False)